__Short summary__

In this notebook:
- [logreg result on 1 feature_metrics calculated WITH roc thr selection (balanced accuracy, f1-score, mcc)](#logreg_selection_roc);
- [logreg result on 1 feature_metrics calculated without roc thr selection](#without_roc_thr);
- [rule-based selection](#rule_based_thrs).

Result:  

CSV table with log_reg mcc and acc for each feature that can be used for further feature selection.



# LogRegression Threshold best based on ROC curve (balanced acc)

Для каждой фичи - LogReg на in_domain_train, затем - по max balanced Accuracy определяется оптимальный порог по roc-кривой. Найденный порог используется для классификации предложений.

In [ ]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
from tqdm.notebook import tqdm
from collections import defaultdict

___
# Settings

In [ ]:
subset =  "in_domain_train" 
model_name =   "nt"
tables_dir = './tables/'

In [ ]:
features_values = pd.read_csv(f"{tables_dir}all_features_{subset}_{model_name}.csv")
features_metrics = pd.read_csv(f"{tables_dir}{subset}_{model_name}.csv")
df = pd.read_csv("./data/cola_public/raw/" + subset + ".tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
y = df['label'].values
df.head(2)

In [ ]:
df_in_domain = pd.read_csv("./data/cola_public/raw/in_domain_dev.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
X_in_domain = pd.read_csv(f"{tables_dir}all_features_in_domain_dev_{model_name}.csv")
y_in_domain = df_in_domain['label'].values
X_in_domain.head(2)

In [ ]:
df_out_domain = pd.read_csv("./data/cola_public/raw/out_of_domain_dev.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
y_out_domain = df_out_domain['label'].values
X_out_domain = pd.read_csv(f"{tables_dir}all_features_out_of_domain_dev_{model_name}.csv")
X_out_domain.head(2)

___

In [ ]:
def confusion_matrix_curve(y, yhat):
    p = np.sum(y)
    n = y.shape[0] - p
    fpr, tpr, threshold = roc_curve(y, yhat)
    tns = []
    fps = []
    fns = []
    tps = []
    for i in range(len(fpr)):
        fp = fpr[i] * n
        tp = tpr[i] * p
        tn = n - fp
        fn = p - tp
        tns.append(tn)
        fps.append(fp)
        fns.append(fn)
        tps.append(tp)
    return np.array(tns), np.array(fps), np.array(fns), np.array(tps), threshold
def find_best_threshold(y, yhat, scoring='balanced_accuracy'):
    if scoring == 'balanced_accuracy':
        fpr, tpr, threshold = roc_curve(y, yhat)
        thr = threshold[np.argmax(tpr - fpr)] # max. tpr - fpr => max. balanced accuracy
        y_pred = np.array([1 if i > thr else 0 for i in yhat])
    elif scoring == 'matthews_corrcoef':
        tns, fps, fns, tps, threshold = confusion_matrix_curve(y, yhat)
        mccs = compute_mcc(tns, fps, fns, tps)
        thr = threshold[np.argmax(mccs)]
        y_pred = np.array([1 if i > thr else 0 for i in yhat])
    else:
        print('Unknown scoring', file=sys.stderr)
        assert 0
    return thr, y_pred

def compute_mcc(tns, fps, fns, tps):
    a = []
    for i in range(len(tns)):
        top = tps[i] * tns[i] - fps[i] * fns[i]
        if top == 0:
            a.append(0)
        else:
            bottom = np.sqrt((tps[i] + fps[i]) * (tps[i] + fns[i]) * (tns[i] + fps[i]) * (tns[i] + fns[i]))
            a.append(top / bottom)
    return np.array(a)

In [18]:
# Список фич отсортированных по P_vaue
features_metrics = features_metrics.sort_values(by = ['p_value']).reset_index(drop = True).copy()

In [19]:
features_metrics.head(2)

,feature,type,corr,p_value
0,v_t0.75_8_6,topological,0.254270,1.684520e-120
1,v_t0.1_8_6,topological,-0.253402,4.047799e-118


<a id='logreg_selection_roc'></a>

In [75]:
def log_reg_thr_search(metrics_dict, scoring_funsction_name = 'balanced_accuracy', roc_thr_search = True):
    for i in tqdm(range(features_metrics.shape[0])): # features_metrics - отсортированный список фич по p_value
    
        feature_ = features_metrics.feature.values[i] # один i-ый признак
        X = features_values.loc[:, feature_].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                            test_size=0.25, random_state=1)
        model = LogisticRegression(max_iter=10000)
        model.fit(X_train.reshape(-1, 1), y_train)

        yhat = model.predict_proba(X_test.reshape(-1, 1))
        yhat = yhat[:, 1]
        if roc_thr_search:
            best_thresh,_ = find_best_threshold(y_test, yhat, scoring = scoring_funsction_name)
            
        # ГРАФИК ROC с лучшим значением ROC

        # # plot the roc curve for the model
        # plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
        # plt.plot(fpr, tpr, marker='.', label='Logistic')
        # plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')
        # # axis labels
        # plt.xlabel('False Positive Rate')
        # plt.ylabel('True Positive Rate')
        # plt.legend()
        # # show the plot
        # plt.show()

        acc, mcc = 0, 0
        skf = StratifiedKFold(n_splits=3, random_state = 1, shuffle=True, )
        skf.get_n_splits(X, y)  

        acc_str, mcc_str = [], []
        acc_in_domain, mcc_in_domain = [], []
        acc_out_domain, mcc_out_domain = [], []
        list_sets = [(X_in_domain.loc[:, feature_].values, y_in_domain, acc_in_domain, mcc_in_domain),
                    (X_out_domain.loc[:, feature_].values, y_out_domain, acc_out_domain, mcc_out_domain)]
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            model_str = LogisticRegression(max_iter=10000)
            model_str.fit(X_train.reshape(-1, 1), y_train)
            yhat, y_pred = [], []
            
            if roc_thr_search:
                yhat = model_str.predict_proba(X_test.reshape(-1, 1))
                yhat = yhat[:, 1]
                y_pred = np.array([1 if i>best_thresh else 0 for i in yhat])
            else:
                y_pred = model_str.predict(X_test.reshape(-1, 1))
            acc, mcc = accuracy_score(y_test, y_pred), matthews_corrcoef(y_test, y_pred)
            acc_str.append(acc)
            mcc_str.append(mcc)
            for sets in list_sets:
                X_test_, y_test_, aucs_, mccs_ = sets
                yhat = model_str.predict_proba(X_test_.reshape(-1, 1))
                yhat = yhat[:, 1]
                y_pred = np.array([1 if i>best_thresh else 0 for i in yhat])
                acc, mcc = accuracy_score(y_test_, y_pred), matthews_corrcoef(y_test_, y_pred)
                aucs_.append(acc)
                mccs_.append(mcc)
        for key, list_ in zip(['in_train_acc', 'in_dev_acc', 'out_dev_acc'], 
                              [acc_str, acc_in_domain, acc_out_domain]):
            metrics_dict[key].append(np.mean(list_))
        for key, list_ in zip(['in_train_mcc', 'in_dev_mcc', 'out_dev_mcc'], 
                              [mcc_str, mcc_in_domain, mcc_out_domain]):
            metrics_dict[key].append(np.mean(list_))
    return metrics_dict

In [76]:
# для константных предсказаний ошибка при расчете MCC
warnings.filterwarnings("ignore", category=RuntimeWarning) 
metrics_dict = log_reg_thr_search(defaultdict(list), scoring_funsction_name = 'balanced_accuracy')

In [77]:
metrics_dict.keys()

dict_keys(['in_train_acc', 'in_dev_acc', 'out_dev_acc', 'in_train_mcc', 'in_dev_mcc', 'out_dev_mcc'])

In [ ]:
for i in metrics_dict.keys():
    features_metrics[i+'_roc_acc'] = metrics_dict[i]

**Max MCCs and accuracies**

In [ ]:
max(metrics_dict['in_train_mcc']), max(metrics_dict['in_dev_mcc']), max(metrics_dict['out_dev_mcc'])

In [ ]:
max(metrics_dict['in_dev_acc']), max(metrics_dict['out_dev_acc']),  max(metrics_dict['in_train_acc'])

# Результаты без отбора thr по ROC кривой
<a id='without_roc_thr'></a>

In [34]:
metrics_dict = log_reg_thr_search(defaultdict(list), scoring_funsction_name = 'balanced_accuracy', roc_thr_search = False)

In [35]:
max(metrics_dict['in_train_mcc']), max(metrics_dict['in_dev_mcc']), max(metrics_dict['out_dev_mcc'])

(0.19352888608236193, 0.1676392461660993, 0.20722752599958982)

In [36]:
max(metrics_dict['in_dev_acc']), max(metrics_dict['out_dev_acc']),  max(metrics_dict['in_train_acc'])

(0.7065148640101202, 0.7093023255813954, 0.7136010551340353)

# Threshold best based on Precision-recall curve 

- Bad results

In [39]:
#@title Bad results
# thresholds = []
# metrics_dict = defaultdict(list)
# for i in tqdm(range(features_metrics.shape[0])):
#     feature_ = features_metrics.feature.values[i]
#     X = features_values.loc[:, feature_].values
#     X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                         test_size=0.25, random_state=1)
#     model = LogisticRegression(max_iter=10000, random_state = 1)
#     # fit a model
#     model.fit(X_train.reshape(-1, 1), y_train)
#     # predict probabilities
#     yhat = model.predict_proba(X_test.reshape(-1, 1))
#     # keep probabilities for the positive outcome only
#     yhat = yhat[:, 1]
#     # calculate roc curves
#     precision, recall, threshold = precision_recall_curve(y_test, yhat)
#     fscore = (2 * precision * recall) / (precision + recall)
#     # locate the index of the largest f score
#     ix = np.argmax(fscore)
#     # J = tpr - fpr
#     # ix = np.argmax(J)
#     best_thresh = threshold[ix]
#     thresholds.append(best_thresh)
#     acc, mcc = 0, 0
#     skf = StratifiedKFold(n_splits=3, random_state = 1, shuffle=True, )
#     skf.get_n_splits(X, y)  
    
#     acc_str, mcc_str = [], []
#     acc_in_domain, mcc_in_domain = [], []
#     acc_out_domain, mcc_out_domain = [], []
#     list_sets = [(X_in_domain.loc[:, feature_].values, y_in_domain, acc_in_domain, mcc_in_domain),
#                 (X_out_domain.loc[:, feature_].values, y_out_domain, acc_out_domain, mcc_out_domain)]
#     for train_index, test_index in skf.split(X, y):
#         X_train, X_test = X[train_index], X[test_index]
#         y_train, y_test = y[train_index], y[test_index]
#         model_str = LogisticRegression(max_iter=10000)
#         model_str.fit(X_train.reshape(-1, 1), y_train)
#         yhat = model.predict_proba(X_test.reshape(-1, 1))
#         yhat = yhat[:, 1]
#         y_pred = np.array([1 if i>best_thresh else 0 for i in yhat])
#         acc, mcc = 0, 0
#         if np.unique(y_pred).shape[0]>1:
#             acc, mcc = accuracy_score(y_test, y_pred), matthews_corrcoef(y_test, y_pred)
#         acc_str.append(acc)
#         mcc_str.append(mcc)
#         for sets in list_sets:
#             X_test_, y_test_, aucs_, mccs_ = sets
#             yhat = model.predict_proba(X_test_.reshape(-1, 1))
#             yhat = yhat[:, 1]
#             y_pred = np.array([1 if i>best_thresh else 0 for i in yhat])
#             acc, mcc = 0, 0
#             if np.unique(y_pred).shape[0]>1:
#                 acc, mcc = accuracy_score(y_test_, y_pred), matthews_corrcoef(y_test_, y_pred)
#             aucs_.append(acc)
#             mccs_.append(mcc)

#     aucs.append(np.mean(acc_in_domain))
#     for key, list_ in zip(['in_train_acc', 'in_dev_acc', 'out_dev_acc'], 
#                           [acc_str, acc_in_domain, acc_out_domain]):
#         metrics_dict[key].append(np.mean(list_))


#     for key, list_ in zip(['in_train_mcc', 'in_dev_mcc', 'out_dev_mcc'], 
#                           [mcc_str, mcc_in_domain, mcc_out_domain]):
#         metrics_dict[key].append(np.mean(list_))

In [40]:
# for i in metrics_dict.keys():
#     features_metrics[i+'_prec'] = metrics_dict[i]

# Rule-based GridSearch features' thrs

<a id='rule_based_thrs'></a>

Поиск rule-based порогов для **значений** признака:

(if feature value>=thr predicted_value = 1 else 0)

In [41]:
class Basic_classifier_thr(BaseEstimator, ClassifierMixin):
    # basic constructor containing threshold param
    def __init__(self, thr=0.72):
        self.thr = thr
    # fitting X - np.array of features'values
    # and y - target feature's values
    def fit(self, X, y):
        self.X_ = X
        self.y_ = y
        return self

    def predict(self, X):
        y_pred = []
        parameter = self.thr
        for x in X:
            if x > parameter:
                y_pred.append(1)
            else:
                y_pred.append(0)
        y_pred = np.array(y_pred)
        return y_pred

In [42]:
metrics_dict_rule = defaultdict(list)
for i in tqdm(range(features_metrics.shape[0])):
    feature_ = features_metrics.sort_values(by = ['p_value']).feature.values[i]
    X_train, X_test, y_train, y_test = train_test_split(features_values.loc[:, feature_].values, y, 
                                                        test_size=0.25, random_state=1)
    mi, ma = features_values.loc[:, feature_].values.min(), features_values.loc[:, feature_].values.max()
    grid_search = GridSearchCV(Basic_classifier_thr(), {'thr': np.linspace(mi, ma,100)})
    grid_search.fit(X_train, y_train)
    best_thresh = grid_search.best_estimator_.thr
    y_pred = np.array([1 if i> best_thresh else 0 for i in X_test])
    # if feature is bigger than that thr - 1 predicted
    # acc, mcc = 0, 0
    skf = StratifiedKFold(n_splits=3, random_state = 1, shuffle=True, )
    skf.get_n_splits(X, y)  
    
    acc_str, mcc_str = [], []
    acc_in_domain, mcc_in_domain = [], []
    acc_out_domain, mcc_out_domain = [], []
    list_sets = [(X_in_domain.loc[:, feature_].values, y_in_domain, acc_in_domain, mcc_in_domain),
                (X_out_domain.loc[:, feature_].values, y_out_domain, acc_out_domain, mcc_out_domain)]
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        ypred = np.array([1 if i>best_thresh else 0 for i in X_test])
        acc, mcc = accuracy_score(y_test, ypred), matthews_corrcoef(y_test, ypred)
        acc_str.append(acc)
        mcc_str.append(mcc)
        for sets in list_sets:
            X_test_, y_test_, aucs_, mccs_ = sets
            y_pred = np.array([1 if i>best_thresh else 0 for i in X_test_])
            acc, mcc = accuracy_score(y_test_, y_pred), matthews_corrcoef(y_test_, y_pred)
            aucs_.append(acc)
            mccs_.append(mcc)

    for key, list_ in zip(['in_train_acc', 'in_dev_acc', 'out_dev_acc'], 
                          [acc_str, acc_in_domain, acc_out_domain]):
        metrics_dict_rule[key].append(np.mean(list_))
    for key, list_ in zip(['in_train_mcc', 'in_dev_mcc', 'out_dev_mcc'], 
                          [mcc_str, mcc_in_domain, mcc_out_domain]):
        metrics_dict_rule[key].append(np.mean(list_))

In [43]:
for i in metrics_dict_rule.keys():
    features_metrics[i+'_rule'] = metrics_dict_rule[i]

In [ ]:
max(metrics_dict_rule['in_train_mcc']), max(metrics_dict_rule['in_dev_mcc']), max(metrics_dict_rule['out_dev_mcc'])

In [ ]:
max(metrics_dict_rule['in_dev_acc']), max(metrics_dict_rule['out_dev_acc']),  max(metrics_dict_rule['in_train_acc'])

# LogRegression Threshold best based on ROC curve (f1-score)

Threshold selection based on roc f1-score best

In [45]:
metrics_dict = defaultdict(list)
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')
for i in tqdm(range(features_metrics.shape[0])):
    
    feature_ = features_metrics.feature.values[i]
    X = features_values.loc[:, feature_].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.25, random_state=1)
    model = LogisticRegression(max_iter=10000)
    model.fit(X_train.reshape(-1, 1), y_train)
    yhat = model.predict_proba(X_test.reshape(-1, 1))
    yhat = yhat[:, 1]

    threshold = np.arange(0, 1, 0.01)
    scores = [f1_score(y_test, to_labels(yhat, t)) for t in threshold]
    ix = np.argmax(scores)
    best_thresh = threshold[ix]
    
    acc, mcc = 0, 0
    skf = StratifiedKFold(n_splits=3, random_state = 1, shuffle=True, )
    skf.get_n_splits(X, y)  
    
    acc_str, mcc_str = [], []
    acc_in_domain, mcc_in_domain = [], []
    acc_out_domain, mcc_out_domain = [], []
    list_sets = [(X_in_domain.loc[:, feature_].values, y_in_domain, acc_in_domain, mcc_in_domain),
                (X_out_domain.loc[:, feature_].values, y_out_domain, acc_out_domain, mcc_out_domain)]
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model_str = LogisticRegression(max_iter=10000)
        model_str.fit(X_train.reshape(-1, 1), y_train)

        yhat = model.predict_proba(X_test.reshape(-1, 1))
        yhat = yhat[:, 1]
        y_pred = np.array([1 if i>=best_thresh else 0 for i in yhat])
        acc, mcc = accuracy_score(y_test, y_pred), matthews_corrcoef(y_test, y_pred)
        acc_str.append(acc)
        mcc_str.append(mcc)
        for sets in list_sets:
            X_test_, y_test_, aucs_, mccs_ = sets
            yhat = model.predict_proba(X_test_.reshape(-1, 1))
            yhat = yhat[:, 1]
            y_pred = np.array([1 if i>=best_thresh else 0 for i in yhat])
            acc, mcc = accuracy_score(y_test_, y_pred), matthews_corrcoef(y_test_, y_pred)
            aucs_.append(acc)
            mccs_.append(mcc)
    for key, list_ in zip(['in_train_acc', 'in_dev_acc', 'out_dev_acc'], 
                          [acc_str, acc_in_domain, acc_out_domain]):
        metrics_dict[key].append(np.mean(list_))
    for key, list_ in zip(['in_train_mcc', 'in_dev_mcc', 'out_dev_mcc'], 
                          [mcc_str, mcc_in_domain, mcc_out_domain]):
        metrics_dict[key].append(np.mean(list_))

In [46]:
max(metrics_dict['in_train_mcc']), max(metrics_dict['in_dev_mcc']), max(metrics_dict['out_dev_mcc'])

(0.13504703569611196, 0.1863589434285585, 0.22734261691467486)

In [47]:
max(metrics_dict['in_dev_acc']), max(metrics_dict['out_dev_acc']),  max(metrics_dict['in_train_acc'])

(0.7077798861480075, 0.7131782945736435, 0.7134838089025909)

In [48]:
for i in metrics_dict.keys():
    features_metrics[i+'_grid_log'] = metrics_dict[i]

In [49]:
features_metrics[i+'thr_grid_log'] = thresholds

In [ ]:
# features_metrics.to_csv(f'{tables_dir}features_importances_2.csv')# with rule-based classes

# LogRegression Threshold best based on ROC curve (mcc)

Threshold selection based on roc-curve mcc-score best

In [51]:
metrics_dict = log_reg_thr_search(defaultdict(list), scoring_funsction_name = 'matthews_corrcoef')

In [52]:
max(metrics_dict['in_train_mcc']), max(metrics_dict['in_dev_mcc']), max(metrics_dict['out_dev_mcc'])

(0.21903088976325835, 0.25561809104304334, 0.26401546525398606)

In [53]:
max(metrics_dict['in_dev_acc']), max(metrics_dict['out_dev_acc']),  max(metrics_dict['in_train_acc'])

(0.7096774193548386, 0.7125322997416021, 0.7126650954522985)

In [54]:
for i in metrics_dict.keys():
    features_metrics[i+'_roc_mcc'] = metrics_dict[i]

In [58]:
features_metrics = features_metrics[['feature', 'type', 'p_value', 'corr',
          'in_train_acc_roc_mcc', 'in_dev_acc_roc_mcc', 'out_dev_acc_roc_mcc',
       'in_train_mcc_roc_mcc', 'in_dev_mcc_roc_mcc', 'out_dev_mcc_roc_mcc']]

In [59]:
features_metrics.to_csv(f'{tables_dir}features_importances.csv')# without rule-based classes
features_metrics

,feature,type,p_value,corr,in_train_acc_roc_mcc,in_dev_acc_roc_mcc,out_dev_acc_roc_mcc,in_train_mcc_roc_mcc,in_dev_mcc_roc_mcc,out_dev_mcc_roc_mcc
0,v_t0.75_8_6,topological,1.684520e-120,0.254270,0.680038,0.675522,0.673127,0.219031,0.200834,0.232840
1,v_t0.1_8_6,topological,4.047799e-118,-0.253402,0.691966,0.676154,0.698320,0.202288,0.164001,0.242049
2,h0_t_d_8_6,barcode,4.543650e-107,-0.236393,0.563562,0.574320,0.558786,0.208765,0.226206,0.240258
3,b1_t0.1_8_6,topological,4.863425e-95,-0.200544,0.582738,0.577483,0.569121,0.195701,0.183921,0.217593
4,v_t0.25_8_6,topological,2.697872e-93,-0.228246,0.638523,0.641366,0.653101,0.203697,0.193526,0.264015
...,...,...,...,...,...,...,...,...,...,...
8923,h1_n_b_l_t0.70_4_4,barcode,9.992989e-01,-0.002067,0.295638,0.307400,0.313953,0.000000,0.000000,0.000000
8924,h1_nb_0_2,barcode,9.996988e-01,-0.009926,0.659691,0.653384,0.639535,-0.005526,-0.002186,0.006828
8925,h1_n_b_m_t0.25_0_2,barcode,9.996988e-01,-0.009926,0.659691,0.653384,0.639535,-0.005526,-0.002186,0.006828
8926,h0_e_5_2,barcode,9.997128e-01,-0.004767,0.683200,0.674889,0.654393,0.012305,0.010772,0.001864
